<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2021/homework4-influxdb/blob/main/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install the required packages
!pip3 install influxdb plotly pandas numpy matplotlib

In [ ]:
from influxdb import InfluxDBClient
import json
import datetime
from getpass import getpass
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib.pylab import rcParams
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# following commands are to set the plotly figures.

%matplotlib inline

sns.set(style='whitegrid', palette='muted')
rcParams['figure.figsize'] = 14, 8

In [ ]:
#ensure client object is not null. this will only happen if the connection succeeded.
passwd=getpass('Enter Password Here:')
client = InfluxDBClient(host='replace with the public IP of your ec2 instance', username='admin', password='your db password', port=8086, database="energy")

Enter Password Here:··········


In [ ]:
# Assuming you have created the energy database through influxd CLI. 
# Now, test the connection status...
print('existing databases:', client.get_list_database())

# create an example database
client.create_database('example')
print('existing databases:', client.get_list_database())

# delete the example database
client.drop_database('example')
print('existing databases:', client.get_list_database())

# You should be able to see the following outputs after executing this cell:

"""
existing databases: [{'name': '_internal'}, {'name': 'energy'}]
existing databases: [{'name': '_internal'}, {'name': 'energy'}, {'name': 'example'}]
existing databases: [{'name': '_internal'}, {'name': 'energy'}]
"""

## An example of plotting synthetic data using plotly library.

Plotly allows you to test and make simple figures.


In [ ]:
# define the time range
begin = datetime.datetime(2021, 1, 1)
end = datetime.datetime(2021, 1, 3, 23, 59, 59)
step = datetime.timedelta(hours=1) # set sampling step as 1 hour

# generate X and Y axises
timesteps = []
values = []
while begin < end:
    timesteps.append(begin.strftime('%Y-%m-%d %H:%M:%S'))
    begin += step
    values.append(np.random.randint(0, 100))

# plot the figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=timesteps, y=values, mode='lines', name='Synthetic Values'))
fig.update_layout(showlegend=True)
fig.show()

# Task1: Import the energy data from CSV file to remote InfluxDB -- 15 points

In [ ]:
def load_data():
    """
    TODO: Load the energy dataset into InfluxDB database you created manually.
    hints: 
    1. use write_points function
    2. no tag needs to be set
    3. set "date" as time.
    """
    pass

# Task2: Queries
## Some Hints:
1. Since the sampling frequency of the given dataset is 10 mins, in the following queries, you usually need to use the "GROUP BY(1d)" statement to get the daily average value of a field before doing other operations.
2. Try to use some built-in InfluxQL functions when solving the problems, rather than reinventing the wheel. Please refer https://docs.influxdata.com/influxdb/v1.8/query_language/functions/#spread.
3. To pass the provided testing assertion, you may need to round your answer to floating numbers with 2 decimal places.
4. You can refer figures in the images folder to verify your results.
5. You are allowed to use pandas or numpy to process query results to make it match provided validation result
6. Each assert statement below is effectively a test.

### Q1: Calculate the mean, median, and standard deviation of each field and return a DataFrame. --  This query has been solved as an example

In [ ]:
def q1():
    """
    :return: DataFrame
    """
    query_str = 'SELECT MEAN(*), MEDIAN(*), STDDEV(*) FROM energy FILL(none)'
    result = list(client.query(query_str).get_points('energy'))[0]
    fileds = list(set(['_'.join(item.split('_')[1:]) for item in list(result.keys())[1:]]))
    df_rlt = {}
    for f in fileds:
        df_rlt.update({f: {'mean': result['mean_%s'%f], 'median': result['median_%s'%f], 'stddev': result['stddev_%s'%f]}})
    return pd.DataFrame(df_rlt).T.round(2)

In [ ]:
aq1 = pd.DataFrame({'mean': {'T4': 20.86, 'T_out': 7.41, 'T2': 20.34, 'RH_5': 50.95, 'Windspeed': 4.04, 'lights': 3.8, 'T9': 19.49, 'Press_mm_hg': 755.52, 'T1': 21.69, 'T8': 22.03, 'RH_7': 35.39, 'RH_out': 79.75, 'RH_4': 39.03, 'RH_8': 42.94, 'RH_2': 40.42, 'T7': 20.27, 'T6': 7.91, 'Visibility': 38.33, 'RH_6': 54.61, 'Tdewpoint': 3.76, 'RH_9': 41.55, 'T3': 22.27, 'rv1': 24.99, 'RH_3': 39.24, 'T5': 19.59, 'Appliances': 97.69, 'RH_1': 40.26, 'rv2': 24.99}, 'median': {'T4': 20.67, 'T_out': 6.92, 'T2': 20.0, 'RH_5': 49.09, 'Windspeed': 3.67, 'lights': 0.0, 'T9': 19.39, 'Press_mm_hg': 756.1, 'T1': 21.6, 'T8': 22.1, 'RH_7': 34.86, 'RH_out': 83.67, 'RH_4': 38.4, 'RH_8': 42.38, 'RH_2': 40.5, 'T7': 20.03, 'T6': 7.3, 'Visibility': 40.0, 'RH_6': 55.29, 'Tdewpoint': 3.43, 'RH_9': 40.9, 'T3': 22.1, 'rv1': 24.9, 'RH_3': 38.53, 'T5': 19.39, 'Appliances': 60.0, 'RH_1': 39.66, 'rv2': 24.9}, 'stddev': {'T4': 2.04, 'T_out': 5.32, 'T2': 2.19, 'RH_5': 9.02, 'Windspeed': 2.45, 'lights': 7.94, 'T9': 2.01, 'Press_mm_hg': 7.4, 'T1': 1.61, 'T8': 1.96, 'RH_7': 5.11, 'RH_out': 14.9, 'RH_4': 4.34, 'RH_8': 5.22, 'RH_2': 4.07, 'T7': 2.11, 'T6': 6.09, 'Visibility': 11.79, 'RH_6': 31.15, 'Tdewpoint': 4.19, 'RH_9': 4.15, 'T3': 2.01, 'rv1': 14.5, 'RH_3': 3.25, 'T5': 1.84, 'Appliances': 102.52, 'RH_1': 3.98, 'rv2': 14.5}})

assert q1().eq(aq1).all(axis=1).all()

### Q2: Calculate(10 points) and plot(5 points) the average daily temperature of the teenager room 2 in February. -- 15 points

In [ ]:
def q2():
    """
    :return: List
    """
    pass

In [ ]:
aq2 = [20.65, 20.59, 21.29, 21.29, 20.65, 20.95, 21.73, 21.97, 22.29, 22.45, 22.63, 22.82, 22.12, 21.61, 21.47, 21.52, 21.35, 20.45, 19.97, 20.4, 21.44, 21.28, 20.88, 21.01, 21.21, 20.34, 21.15, 21.51, 21.51]

assert q2() == aq2

### Q3: Return(10 points) and plot(5 points) 7-day rolling average values of daily mean external humidity. -- 15 points

In [ ]:
# Return(10 points) and plot(5 points) 7-day rolling average values of daily mean external humidity. -- 15 points
def q3():
    """
    :return: List
    """
    pass

In [ ]:
aq3 = [86.93, 85.92, 85.38, 86.09, 85.24, 84.9, 85.96, 87.51, 87.19, 87.46, 87.07, 87.79, 87.28, 85.89, 85.65, 86.5, 87.34, 86.88, 87.73, 89.32, 87.94, 85.21, 83.88, 83.56, 84.87, 85.29, 83.95, 86.07, 88.88, 89.28, 88.0, 86.83, 85.37, 85.54, 85.36, 84.81, 86.48, 87.74, 87.75, 87.48, 86.77, 83.39, 79.82, 77.04, 77.7, 77.26, 77.66, 78.48, 81.3, 85.4, 88.29, 86.78, 85.08, 83.45, 82.21, 81.05, 78.71, 76.44, 75.74, 76.1, 75.76, 77.5, 77.59, 77.95, 78.58, 78.57, 80.84, 82.15, 81.76, 81.31, 81.04, 80.18, 79.67, 77.94, 79.15, 78.11, 78.54, 77.98, 78.65, 79.34, 80.32, 78.25, 78.65, 77.85, 78.4, 76.46, 75.73, 76.61, 77.23, 76.8, 77.86, 78.61, 79.62, 79.55, 76.31, 73.59, 71.73, 70.2, 70.19, 72.86, 74.02, 76.31, 78.73, 80.89, 82.25, 80.29, 76.83, 74.83, 72.12, 68.64, 65.51, 61.27, 59.37, 58.04, 60.55, 62.85, 66.55, 68.52, 70.42, 72.81, 75.22, 73.9, 73.71, 73.23, 74.46, 75.99, 78.54, 79.9, 79.96, 79.67, 79.57, 78.48]

assert q3() == aq3

### Q4: Obtain the number of days when the absolute difference in Appliances energy consumption between adjacent dates is greater than 100Wh. -- 10 points

In [ ]:
def q4():
    """
    :return: int
    """
    pass

In [ ]:
assert q4() == 3

### Q5: Calculate the difference between the maximum and minimum daily mean Appliances energy consumption in March. -- 10 points

In [ ]:
def q5():
    """
    :return: float
    """
    pass

In [ ]:
assert q5() == 117.78

### Q6: It is known that the daily average humidity outside the building (north side , column name: RH_6) has varying degrees of negative correlation with the daily mean temperature of rooms. Please determine which room’s temperature is most affected by this humidity (ie the smallest negcorrelation coefficient). Return the correlation coefficient and plot daily average curves for RH_6 and the field you identified. -- 15 points

Hint: You may use the corr function in pandas to calculate correlation coefficients between DataFrame columns. see https://www.w3resource.com/pandas/dataframe/dataframe-corr.php 

In [ ]:
def q6():
    """
    :return: float
    """
    pass

In [ ]:
assert q6() == -0.82